In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
trains='/kaggle/input/imge-classification-animal/animal-classification/train/'
sub=pd.read_csv('/kaggle/input/imge-classification-animal/Sample_submission.csv')

In [ ]:
tests_paths = []
for i in range(1, 730):
    tests_paths.append('/kaggle/input/imge-classification-animal/animal-classification/test/test/' + str(i) + '.jpg')
test_data = {'path': tests_paths, 'class': '0'}
df = pd.DataFrame(test_data)
df.head()

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
image_data = ImageDataGenerator(rescale=1/255, validation_split=0.1)
test_datagen = ImageDataGenerator(rescale=1/255)

BATCH_SIZE = 16
input_shape = (224, 224)

train_data = image_data.flow_from_directory(trains, target_size = input_shape,
    batch_size = BATCH_SIZE, class_mode = 'categorical', subset='training')
valid_data = image_data.flow_from_directory(trains, target_size = input_shape,
    batch_size = BATCH_SIZE, class_mode = 'categorical', subset='validation')
test_data = test_datagen.flow_from_dataframe(df, x_col='path', y_col='class', target_size = input_shape,
    batch_size = BATCH_SIZE, shuffle=False)

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout, BatchNormalization
from tensorflow.keras import applications
model = applications.VGG16(weights='imagenet', include_top=False,  input_shape=(224, 224, 3))

In [ ]:
# https://towardsdatascience.com/fine-tuning-pre-trained-model-vgg-16-1277268c537f
for layer in model.layers[:15]:
    layer.trainable = False
for i, layer in enumerate(model.layers):
    print(i, layer.name, layer.trainable)

In [ ]:
# https://towardsdatascience.com/fine-tuning-pre-trained-model-vgg-16-1277268c537f
x = model.output
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.25)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.25)(x)
x = Dense(64, activation='relu')(x)
x = Dense(16, activation='relu')(x)
x = Dense(4, activation='softmax')(x)
transfer_model = Model(inputs=model.input, outputs=x)
transfer_model.summary()

In [ ]:
from keras.callbacks import ReduceLROnPlateau
lr_reduce = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, verbose=1, mode='min', min_lr=0)

In [ ]:
from tensorflow.keras import layers, models, Model, optimizers
transfer_model.compile(loss="categorical_crossentropy", optimizer=optimizers.Adam(lr=0.00005), metrics=["acc"])
hist = transfer_model.fit(train_data, epochs=50, validation_data=valid_data, callbacks=[lr_reduce])

In [ ]:
import matplotlib.pyplot as plt
loss = hist.history['loss']
val_loss = hist.history['val_loss']
epoch = np.arange(1, len(loss) + 1)
plt.figure(figsize=(10, 8))
plt.plot(epoch, loss, 'b', label='loss')
plt.plot(epoch, val_loss, 'g', label='val_loss')
plt.title('Loss')
plt.legend()

In [ ]:
acc = hist.history['acc']
val_acc = hist.history['val_acc']
epoch = np.arange(1, len(loss) + 1)
plt.figure(figsize=(10, 8))
plt.plot(epoch, loss, 'b', label='acc')
plt.plot(epoch, val_loss, 'g', label='val_acc')
plt.title('Accuracy')
plt.legend()

In [ ]:
pre = transfer_model.predict(test_data)
pre = pre.argmax(1)

In [ ]:
pred = pd.DataFrame(pre)
sub['Label'] = pre
sub.to_csv('./submission.csv', index = False)